In [1]:
import awkward as ak
import numpy as np
import os, json, time, uproot, yaml, correctionlib

from coffea import processor, lumi_tools
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.nanoaod import FatJetArray
from coffea.analysis_tools import PackedSelection

import sys
sys.path.append("..")
from src.processors.Processor import Processor
from src.processors.GenMatch import GenMatch

/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-setuptools/60.9.3-ddcab3b6a742c72fed5554de8ccaaf18/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-setuptools/60.9.3-ddcab3b6a742c72fed5554de8ccaaf18/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/cvmfs/cms.cern.ch/slc7_amd64_gcc10/external/py3-numpy/1.22.4-fed14fba556da687b0d60c63c90ca3bc/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarn

# Batch test

In [2]:
uproot.open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource
_events = NanoEventsFactory.from_root(
    #file='https://stash.osgconnect.net/+dfu/customizedNanoAOD/output/mc/2018/GJets/GJets_HT-200To400/1397.root',
    file='13.root',
    treepath='Events', schemaclass=NanoAODSchema
).events()
p=Processor(outdir='./test/', mode='mc_2017_ZpToHG', param_dir='../src/parameters/')
c=p.process(_events)


In [3]:
ak.mean(p.variables['nExtraAK4jet']), ak.mean(p.variables['nAK4jet'])

(3.8683870967741933, 6.172903225806452)

In [4]:
p.variables['nExtraAK4jet'], p.variables['nAK4jet']

(<Array [2, 6, 6, 2, 5, 4, ... 5, 1, 2, 1, 3, 3] type='775 * ?int64'>,
 <Array [4, 10, 8, 4, 7, 6, ... 3, 4, 4, 5, 7] type='775 * ?int64'>)

In [5]:
_events.FatJet.fields

['ParticleNetMDraw_probQCDb',
 'ParticleNetMDraw_probQCDbb',
 'ParticleNetMDraw_probQCDc',
 'ParticleNetMDraw_probQCDcc',
 'ParticleNetMDraw_probQCDothers',
 'ParticleNetMDraw_probXbb',
 'ParticleNetMDraw_probXcc',
 'ParticleNetMDraw_probXqq',
 'ParticleNetraw_probHbb',
 'ParticleNetraw_probHcc',
 'ParticleNetraw_probHqqqq',
 'ParticleNetraw_probQCDb',
 'ParticleNetraw_probQCDbb',
 'ParticleNetraw_probQCDc',
 'ParticleNetraw_probQCDcc',
 'ParticleNetraw_probQCDothers',
 'ParticleNetraw_probTbc',
 'ParticleNetraw_probTbcq',
 'ParticleNetraw_probTbel',
 'ParticleNetraw_probTbmu',
 'ParticleNetraw_probTbq',
 'ParticleNetraw_probTbqq',
 'ParticleNetraw_probTbta',
 'ParticleNetraw_probWcq',
 'ParticleNetraw_probWqq',
 'ParticleNetraw_probZbb',
 'ParticleNetraw_probZcc',
 'ParticleNetraw_probZqq',
 'area',
 'btagCMVA',
 'btagCSVV2',
 'btagDDBvL',
 'btagDDBvLV2',
 'btagDDBvL_noMD',
 'btagDDCvB',
 'btagDDCvBV2',
 'btagDDCvB_noMD',
 'btagDDCvL',
 'btagDDCvLV2',
 'btagDDCvL_noMD',
 'btagDeepB',


In [ ]:
run = processor.Runner(
        executor=processor.FuturesExecutor(compression=None, workers=20),
        schema=NanoAODSchema,
        savemetrics=True,
        xrootdtimeout=60 * 30,
        # chunksize = 100_000,
        # maxchunks = None,
    )
stats, metrics = run(
    fileset={'input': ['https://stash.osgconnect.net/+fudawei/samples/mc/2018/ZpToHG/ZpToHGamma_M3500/merged.root']},
    treename='Events',
    processor_instance=Processor(mode='mc_2018_ZpToHG', outdir='./test', param_dir='../src/parameters/'),
)

Output()

Output()

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/pku/fudawei/anaconda3/lib/python3.9/site-packages/coffea/processor/executor.py", line 1640, in _work_function
    out = processor_instance.process(events)
  File "/data/pubfs/fudawei/NanoAnalysis/notebook/../src/processors/Processor.py", line 333, in process
    final_cut = self.preselect_HGamma()
  File "/data/pubfs/fudawei/NanoAnalysis/notebook/../src/processors/Processor.py", line 284, in preselect_HGamma
    Higgs_score = ak.sum([self.object['AK8jet'][f] for f in self.event.FatJet.fields if f.startswith('inclParTMDV1_probH')], axis=0, mask_identity=True)
  File "/home/pku/fudawei/anaconda3/lib/python3.9/site-packages/awkward/operations/reducers.py", line 329, in sum
    layout = ak.operations.convert.to_layout(
  File "/home/pku/fudawei/anaconda3/lib/python3.9/site-packages/awkward/operations/convert.py", line 1930, in to_layout
    return from_iter(array, highlevel=False)
  File "/ho

Exception: Failed processing file: WorkItem(dataset='input', filename='https://stash.osgconnect.net/+fudawei/samples/mc/2018/ZpToHG/ZpToHGamma_M3500/merged.root', treename='Events', entrystart=0, entrystop=120000, fileuuid=b'NE0\xd6\xf4r\x11\xed\x89\xcb\x85\x06\xb9\xbc\xbe\xef', usermeta={})

In [19]:
set(ak.flatten(_events.GenPart[_events.Photon.genPartIdx].pdgId))

{-4332,
 -4232,
 -4132,
 -4122,
 -431,
 -421,
 -411,
 -321,
 -311,
 -211,
 -14,
 -13,
 -12,
 -11,
 11,
 12,
 13,
 14,
 21,
 22,
 211,
 321,
 323,
 411,
 421,
 431,
 441,
 443,
 4122,
 4232,
 4332}

In [18]:
set(ak.flatten(_events.GenPart[_events.Photon.genPartIdx[_events.Photon.genPartIdx!=-1]].pdgId))

{-11, 11, 22}

In [15]:
_events.GenPart[0][-1]

GenParticle

In [4]:
ak.argmax(
    [None, None, None], axis=1, keepdims=True, mask_identity=False
)

ValueError: axis=1 exceeds the depth of the nested list structure (which is 1)

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.2/src/libawkward/Content.cpp#L1061)

In [6]:
a=ak.from_parquet('./test/22737784-5359-11ed-a3a4-0245010abeef_Events_0-37337.parq')

In [8]:
a.AK8jet_rankToHiggsMass

<Array [0] type='1 * ?int64'>

In [11]:
run = processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=1),
    schema = NanoAODSchema,
    savemetrics = True,
    xrootdtimeout = 60 * 30,
    #chunksize = 100_000,
    #maxchunks = 1,
)
stats, metrics = run(
    fileset = {'input': ['root://cms-xrd-global.cern.ch//store/data/Run2017B/SinglePhoton/NANOAOD/UL2017_MiniAODv2_NanoAODv9-v1/120000/4DEB63D4-999D-7946-AF29-BB04A0CBC247.root']},
    treename = 'Events',
    processor_instance = Processor(mode='data_2017_SinglePhoton', outdir='./test/', param_dir='../src/parameters/'),
)

SyntaxError: invalid syntax (726123375.py, line 12)

In [10]:
stats

{'data_2017_SinglePhoton': {'n_events': 10026.0,
  'triggered': 1127.0,
  'filtered': 1079.0,
  'photon': 104.0,
  'AK8jet': 24.0,
  '2018_HEM_correction': 24.0,
  'photon-jet_cleaning': 16.0,
  'b-veto': 13.0}}

In [10]:
correction = correctionlib.CorrectionSet.from_file(p.pile_up['json'][p.year])[p.pile_up['key'][p.year]]

In [17]:
_events = NanoEventsFactory.from_root(
    #file='root://cms-xrd-global.cern.ch//store/mc/RunIISummer20UL18NanoAODv9/GJets_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/4DEF4AA9-9736-594E-AB77-3F33D317DDBF.root',
    file = 'https://stash.osgconnect.net/+fudawei/samples/mc/2018/ZpToHG/ZpToHGamma_M2000/0/mc2018.root',
    #file='https://stash.osgconnect.net/+yuzhe/0lepton/custom_nano/V2/2018/HG/QCD_HT300to500_TuneCP5_13TeV-madgraphMLM-pythia8_RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v2/out_101.root',
    treepath='Events', schemaclass=NanoAODSchema
).events()

In [46]:
_events.FatJet[ak.argmax(_events.FatJet.inclParTMDV1_probHWqqWev1c, axis=1)]

<FatJetArray [[FatJet], [FatJet, ... [FatJet]] type='1000 * option[var * fatJet]'>

In [22]:
"""
 'inclParTMDV1_probHWqqWev0c',
 'inclParTMDV1_probHWqqWev1c',
 'inclParTMDV1_probHWqqWmv0c',
 'inclParTMDV1_probHWqqWmv1c',
 'inclParTMDV1_probHWqqWq0c',
 'inclParTMDV1_probHWqqWq1c',
 'inclParTMDV1_probHWqqWq2c',
 'inclParTMDV1_probHWqqWqq0c',
 'inclParTMDV1_probHWqqWqq1c',
 'inclParTMDV1_probHWqqWqq2c',
 'inclParTMDV1_probHWqqWtauev0c',
 'inclParTMDV1_probHWqqWtauev1c',
 'inclParTMDV1_probHWqqWtauhv0c',
 'inclParTMDV1_probHWqqWtauhv1c',
 'inclParTMDV1_probHWqqWtaumv0c',
 'inclParTMDV1_probHWqqWtaumv1c',
 'inclParTMDV1_probHbb',
 'inclParTMDV1_probHcc',
 'inclParTMDV1_probHqq',
 'inclParTMDV1_probHss',
 'inclParTMDV1_probHtauhtaue',
 'inclParTMDV1_probHtauhtauh',
 'inclParTMDV1_probHtauhtaum',
"""

"\n 'inclParTMDV1_probHWqqWev0c',\n 'inclParTMDV1_probHWqqWev1c',\n 'inclParTMDV1_probHWqqWmv0c',\n 'inclParTMDV1_probHWqqWmv1c',\n 'inclParTMDV1_probHWqqWq0c',\n 'inclParTMDV1_probHWqqWq1c',\n 'inclParTMDV1_probHWqqWq2c',\n 'inclParTMDV1_probHWqqWqq0c',\n 'inclParTMDV1_probHWqqWqq1c',\n 'inclParTMDV1_probHWqqWqq2c',\n 'inclParTMDV1_probHWqqWtauev0c',\n 'inclParTMDV1_probHWqqWtauev1c',\n 'inclParTMDV1_probHWqqWtauhv0c',\n 'inclParTMDV1_probHWqqWtauhv1c',\n 'inclParTMDV1_probHWqqWtaumv0c',\n 'inclParTMDV1_probHWqqWtaumv1c',\n 'inclParTMDV1_probHbb',\n 'inclParTMDV1_probHcc',\n 'inclParTMDV1_probHqq',\n 'inclParTMDV1_probHss',\n 'inclParTMDV1_probHtauhtaue',\n 'inclParTMDV1_probHtauhtauh',\n 'inclParTMDV1_probHtauhtaum',\n"

In [3]:
ak.argmax(ak.sum([_events.FatJet[f] for f in _events.FatJet.fields if f.startswith('inclParTMDV1_probH')], axis=0), axis=1)

<Array [0, 1, 1, 1, 1, 1, ... 1, 1, 0, 1, 0, 1] type='1000 * ?int64'>

In [38]:
c=ak.Array([_events.FatJet[f] for f in _events.FatJet.fields if f.startswith('inclParTMDV1_probH')])

In [42]:
ak.argsort(ak.argsort([1.5,2,3,0]))

<Array [1, 2, 3, 0] type='4 * int64'>

In [26]:
correction.evaluate(np.array(_events.Pileup.nPU), 'nominal')

array([0.91870999, 1.00662441, 0.98739476, ..., 1.00883794, 1.16807473,
       1.02087936])

In [3]:
run=processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=1),
    schema = NanoAODSchema,
    savemetrics = True,
    xrootdtimeout = 60 * 30,
    #chunksize = 10,
    #maxchunks = 1,
)
stats, metrics = run(
    fileset = {'input': ['/eos/cms/store/data/Run2016B/SinglePhoton/NANOAOD/ver2_HIPM_UL2016_MiniAODv2_NanoAODv9-v2/2430000/0BC6889E-78AB-D54A-9DAC-DD961CED79FE.root']},
    treename = 'Events',
    processor_instance = Processor(mode='mc_2018_ZpToHG', outdir='./', param_dir='../src/parameters/'),
)

TypeError: __call__() got an unexpected keyword argument 'entrystart'

In [40]:
s = _events.GenPart[
    (abs(_events.GenPart.pdgId) == 9906663) &
    _events.GenPart.hasFlags(["fromHardProcess", "isLastCopy"])
]

In [51]:
_events = NanoEventsFactory.from_root(
    file='root://cms-xrd-global.cern.ch//store/mc/RunIIAutumn18NanoAODv7/ZpToHGamma_M1400_13TeV_TuneCP5_madgraph-pythia8/NANOAODSIM/Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/250000/9FCD7BF6-406A-F34A-9AA9-7790134908D8.root',
    treepath='Events', schemaclass=NanoAODSchema
).events()

In [23]:
p.variables['gen_Zp_mass'][~(ak.num(p.variables['gen_Zp_mass'], axis=1)==1)]

<Array [None, None, None, ... None, None, None] type='456 * option[var * ?float3...'>

In [10]:
for i in p.variables.keys():
    if 'Zp_' in i:
        p.variables[i] = ak.flatten(p.variables[i], axis=1)

In [11]:
p.variables

{'AK8jet_mass': <Array [130, 120, 117, 133, ... 135, 138, 100] type='23185 * ?float32[parameters...'>,
 'AK8jet_phi': <Array [-2.75, 0.247, 1.47, ... 1.59, -2.2] type='23185 * ?float32[parameters={"...'>,
 'AK8jet_eta': <Array [-0.317, -1.09, ... -0.987, 0.951] type='23185 * ?float32[parameters={"__...'>,
 'AK8jet_msoftdrop': <Array [124, 120, 114, 132, ... 112, 136, 99.2] type='23185 * ?float32[parameter...'>,
 'AK8jet_pt': <Array [690, 494, 596, 718, ... 628, 686, 606] type='23185 * ?float32[parameters...'>,
 'photon_phi': <Array [0.361, -2.89, -1.63, ... -1.54, 1.48] type='23185 * ?float32[parameters=...'>,
 'photon_mass': <Array [0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0] type='23185 * ?float32'>,
 'photon_pt': <Array [681, 611, 670, 624, ... 653, 691, 326] type='23185 * ?float32[parameters...'>,
 'photon_eta': <Array [-0.178, -0.185, ... -0.653, -0.839] type='23185 * ?float32[parameters={"...'>,
 'event_MET_pt': <Array [16.4, 46.9, 134, ... 120, 15.6, 186] type='23185 * ?float32[para

In [8]:
t = ak.Array(p.variables)

In [23]:
_events.Photon.electronVeto

<Array [[True, True], [True, ... [], [True]] type='1854856 * var * bool[paramete...'>